# Locomotion Mode Classification Using RNN Architectures

## Imports

In [5]:
import numpy as np
from scipy.io import loadmat
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten
from keras.layers import SimpleRNN, LSTM
import keras.backend as K
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.utils import Sequence, to_categorical

## Read dataset and preprocess

In [ ]:
# Read sit-stand data from file
sit_stand_data = loadmat('../Datasets/ashwin_testdata/sit_stand_data_labeled.mat')
walk_data = loadmat('../Datasets/ashwin_testdata/walk_data_ashwin.mat')

# Some parameters
BATCH_SIZE = 32
SEQUENCE_LENGTH = 200


# Take one 90 seconds of data from the sit-stand and 2 minutes from the walking data for Testing.
tmp_x = sit_stand_data['imu_features']
tmp_y = sit_stand_data['lm_label']
# Sit-Stand test set
X_test = tmp_x[tmp_x.shape[0]-8950:, :]
Y_test = tmp_y[tmp_x.shape[0]-8950:, :]/100.
# Sit-Stand training set
X_train = tmp_x[:tmp_x.shape[0]-8950, :]
Y_train = tmp_y[:tmp_x.shape[0]-8950, :]

tmp_x = walk_data['imu_features']
tmp_y = walk_data['lm_label']/300.

# Total test set
X_test = np.vstack((X_test, tmp_x[tmp_x.shape[0]-12000:, :]))
Y_test = np.vstack((Y_test, tmp_y[tmp_y.shape[0]-12000:, :]))

# Total training set
X_train = np.vstack((X_train, tmp_x[:tmp_x.shape[0]-12000, :]))
Y_train = np.vstack((Y_train, tmp_y[:tmp_x.shape[0]-12000, :]))

print(X_test.shape)
print(X_train.shape)

plt.figure(figsize=(15, 9))
plt.title("Test Set")
plt.plot(X_test[:, 56:])
plt.plot(Y_test*100)
plt.ylim([-120, 120])
plt.show()

# Function to preprocess the data into sequences for the RNN
def get_sub_sequences(data_array, y_array, window_size=120, step_size=90, dims=None, seq_out=False, causal=True):
    rows = data_array.shape[0]
    cols = data_array.shape[1]

    if dims == None:
        outdims = [i for i in range(cols)]
    else:
        outdims = dims
        
    sequences = rows//step_size
    out_x = np.zeros((sequences, window_size, len(outdims)))
    if seq_out:
        out_y = np.zeros((sequences, window_size, y_array.shape[1]))
    else:
        out_y = np.zeros((sequences, y_array.shape[1]))
        
    idxs = range(window_size, rows, step_size)    
    
    for i, j in enumerate(idxs):
        out_x[i, :, :] = data_array[j-window_size:j, outdims]
        if seq_out:
            out_y[i, :, :] = y_array[j-window_size:j, :]
        else:
            out_y[i, :] = y_array[j, :]
    
    return out_x, out_y

# Make a sequence object from the data 
class IMUSeq(Sequence):
    def __init__(self, xin, yin, length=100, stride=1, batch_size=64):
        self.X = xin
        self.Y = yin
        self.stride=stride
        self.length=length
    def __len__(self):
        idxs = [i for i in range(self.length, self.X.shape[0], self.stride)]
        return len(idxs)
    def __getitem__(self, idx):
        #idxs = [i for i in range(idx*self.length, (idx+1)*self.length)]
        x_seq = self.X[idx*self.length:(i+1)*self.length, :]
        y_seq = self.Y[idx*self.length:(i+1)*self.length, :]
train_ts = IMUSeq(X_train, Y_train, length=SEQUENCE_LENGTH)
#train_ts = TimeseriesGenerator(X_train, Y_train, length=SEQUENCE_LENGTH, batch_size=BATCH_SIZE)
#print(train_ts[0])

## Classify using RNN

### Model 1

In [ ]:
# Build model
model1 = Sequential()
model1.add(SimpleRNN(units=6, activation='softmax', 
                     return_sequences=True, 
                     input_shape=(SEQUENCE_LENGTH, X_train.shape[1])))
model1.compile(optimizer='rmsprop', 
               loss='categorical_crossentropy',
               metrics=['accuracy'])
print("Model 1 Summary:")
model1.summary()

# Train model
print("Training:")
model1.fit_generator(train_ts, epochs=30, steps_per_epoch=BATCH_SIZE)